# 基本用法

In [2]:
from dash import Dash, html, dcc, callback, Output, Input
import plotly.express as px
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/gapminder_unfiltered.csv')

app = Dash()

app.layout = [
    html.H1(children='Title of Dash App', style={'textAlign':'center'}),
    dcc.Dropdown(df.country.unique(), 'Canada', id='dropdown-selection'),
    dcc.Graph(id='graph-content')
]

@callback(
    Output('graph-content', 'figure'),
    Input('dropdown-selection', 'value')
)
def update_graph(value):
    dff = df[df.country==value]
    return px.line(dff, x='year', y='pop')

if __name__ == '__main__':
    app.run(debug=True,port=8051)


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run_server.

# 多页面

https://poe.com/s/Fo8q93ySJ46JCA7sSa8t

# 进度条

[这里](https://poe.com/s/ElAzkcZRyofZAlg62s64)描述了如何实现一个实时进度条

## 参考

* [Loading | Dash for Python Documentation | Plotly](https://dash.plotly.com/dash-core-components/loading)

## 示例1

In [3]:
from dash import Dash, dcc, html, Input, Output, callback
import time

app = Dash(__name__)

app.layout = html.Div([
    html.H3("Edit text input to see loading state"),
    html.Div("Input triggers local spinner"),
    dcc.Input(id="loading-input-1"),
    dcc.Loading(
        id="loading-1",
        type="default",
        children=html.Div(id="loading-output-1")
    ),
    html.Div([
        html.Div('Input triggers nested spinner'),
        dcc.Input(id="loading-input-2"),
        dcc.Loading(
            id="loading-2",
            children=[html.Div([html.Div(id="loading-output-2")])],
            type="circle",
        )
    ]),
])


@callback(Output("loading-output-1", "children"), Input("loading-input-1", "value"))
def input_triggers_spinner(value):
    time.sleep(10)
    return value


@callback(Output("loading-output-2", "children"), Input("loading-input-2", "value"))
def input_triggers_nested(value):
    time.sleep(1)
    return value


if __name__ == "__main__":
    app.run(debug=False ,port=8051)


## 示例2

这个例子实现以下效果:

一个按钮,一个表格,点击按钮时使表格处于正在加载中

10秒后加载完成

In [5]:
import dash
from dash import dcc, html, Input, Output
import dash_table
import time

app = dash.Dash(__name__)

# 定义布局
app.layout = html.Div([
    html.Button('Load Table', id='load-button', n_clicks=0),
    dcc.Loading(
        id="loading-table",
        type="default",
        children=dash_table.DataTable(
            id='table',
            columns=[
                {'name': 'Column 1', 'id': 'column1'},
                {'name': 'Column 2', 'id': 'column2'}
            ],
            data=[],
            style_table={'height': '300px', 'overflowY': 'auto'}
        ),
        style={'marginTop': '20px'}
    )
])

# 回调函数
# ! 这个回调中并没有手动更新loading-table的状态,具体解释见 [[这里](https://poe.com/chat/3f6onzy464sl2mku5hu)]
@app.callback(
    Output('table', 'data'),
    Input('load-button', 'n_clicks')
)
def load_data(n_clicks):
    if n_clicks > 0:
        # 模拟一个耗时10秒的操作
        time.sleep(10)
        # 返回一些示例数据
        data = [
            {'column1': 'Data 1-1', 'column2': 'Data 1-2'},
            {'column1': 'Data 2-1', 'column2': 'Data 2-2'},
            {'column1': 'Data 3-1', 'column2': 'Data 3-2'}
        ]
        return data
    return []

if __name__ == '__main__':
    app.run_server(debug=True, port=8051)